<a href="https://colab.research.google.com/github/klarcika/TZ/blob/main/Naloga3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Naloga: Klasifikacija
napovemo podatek, ki je kategoričen


In [ ]:
df=read_csv("nekaj.csv")

In [ ]:
#z datumise ne znamo, zato lahko ven zbrisemo stolpec
df.drop("DateSolf",axis=1, inplace=true)
df.types
#vcasih lahko imamo kategoricen podatek med stevilskim


In [ ]:
!pip install scikit-learn #je ze tu instalirano

In [ ]:
!pip install sklearn-pandas==1.5.0 #je tudi ze

In [1]:
stevilski=["Rooms,..."]

In [ ]:
stevilski=df.select_dtypes(exclude=["object"]).columns
print(stevilski)
kategoricni=df.select_dtypes(exclude=["float64", "int64"]).columns
print(kategoricni)

In [ ]:
#prazne vrednosti lahko nadomestimo
from sklearn_pandas import CategoricalImputer #samo kategoricne
from sklearn.impute import SimpleImputer #za kategoricne in stevilske podatke

In [ ]:

import numpy as np #knjiznica za delo s stevili
povprecje= SimpleImputer(missing_values=np.nan, strategy="mean") #ce dopolnjujemo z stevilskimi, damo most_frequent
df[stevilski]= povprecje.fit_transform(df[stevilski])
#----------------
najpogostejsi= SimpleImputer(missing_values=np.nan, strategy="most_frequent") #ce dopolnjujemo z stevilskimi, damo most_frequent
df[kategoricni]= najpogostejsi.fit_transform(df[kategoricni])


In [ ]:
df.isnull().sum()
#pred klasifikacijo ne smemo met manjkajocih; fajn je uporabit most frequent za kategoricne in povprecje za stevilske, odvisno kolko jih je


In [3]:
#predpogoj : podatke razdelimo na testne in ucne !!
from sklearn.model_selection import train_test_split #y napovedujemo je kategoricen podatek
#dolocimo in izhodne podatke
izhod="Type" #outcome
vhod=["Rooms", "Bathroom","CarSpots", "YearBuilt","Price" ]#predstavlja vec stolpcev, na podlagi cesa klasificiramo? keri so relevantni


x_train,x_test, y_train, y_test= train_test_split(df[vhod], df[izhod], test_size=0.2, random_state=123)#radom state je naključna delitev
x_train.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=5) #določimo st sosedov
knn.fit(x_train, y_train)#damo not ucne podatke
rezultat=knn.predict(x_test)
rezultati[:10]

In [ ]:
#metrike s katerimi previrimo tocnost napovedi
from sklearn import metrics
print(metrics.accuracy_score(rezultat, y_test))

In [5]:
#kfold [[T][U][U][U]] imamo toliko iteracij, dokler nena vsak fold predstavlja testnih podatkov
#s tem dosezemo da so rezultati bolj relevantni; so bolj stabilni in niso podvrzeni nakljucni delitvi
from sklearn.tree import DecisionTreeClassifier

klasifikatorji=[] #kako lahko vec klasifikatorjev naenkrat testiramo
klasifikatorji.append(KNeighborsClassifier())
klasifikatorji.append(DecisionTreeClassifier())
print(len(klasifikatorji))

In [ ]:
from sklearn.model_selection import StratifiedGroupKFold, cross_val_score
for k in klasifikatorji:
  kfold=StratifiedGroupKFold(n_splits=10)
  #stratificiran pomeni ko deli podatke na folde pazi da so izhodni podatki enako porazdeljeni/razmerje kot je v celotni zbirki
  #vsak fold je obetzen na izhodne razrede
  rez=cross_val_score(k, df[vhod], df[izhod], cv=kfold, scoring="accuracy")
print(rez)
#ce zelimo dostopat do imena klasifikatorja uporabimo
print((type(k)._name_, rez.mean())) #povprecje izpisemo, ker potem bi za vsak fold poseaj izpisalo; zato izpisemo povprecje vseh foldov

podatke v tej nalogi si zdruzene podatke nalozimo v csv datoteko; pazi na izhodni/vhodni podatek!!
svc pri enem algoritmu